## 1_pydantic_why

In [6]:
from pydantic import BaseModel, EmailStr, AnyUrl, Field
from typing import List, Dict, Optional, Annotated

class Patient(BaseModel):

    name: Annotated[
        str,
        Field(
            max_length=50,
            title="Name of the patient",
            description="Give the name of the patients in less than 50 chars",
            examples=[{"example_1": "Ramesh"}, {"example_2": "Suresh"}]
        )
    ]
    email: EmailStr
    linkedIn: Optional[AnyUrl] = None
    age: int = Field(gt=0, lt=120)
    weight: Annotated[float, Field(gt=0, strict=True)]
    married: Annotated[bool, Field(default=None, description="Is the patient married or not?")]
    allergies: Annotated[Optional[List[str]], Field(default=None, max_length=5)]
    contact_details: Optional[Dict[str, str]] = {}


def insert_patient_data(patient: Patient):
    print(patient)
    print("Data inserted!")


def update_patient_data(patient: Patient):
    print(patient)
    print("Data updated!")


patient_info = {
    "name": "John",
    "email": "john319@gmail.com",
    "linkedIn": "https://www.linkedin.com/in/john/",
    "age": 30,
    "weight": 70.5,
    "married": True,
    "contact_details": {"phone": "123-456-7890"},
}

patient1 = Patient(**patient_info)

insert_patient_data(patient1)
update_patient_data(patient1)

name='John' email='john319@gmail.com' linkedIn=AnyUrl('https://www.linkedin.com/in/john/') age=30 weight=70.5 married=True allergies=None contact_details={'phone': '123-456-7890'}
Data inserted!
name='John' email='john319@gmail.com' linkedIn=AnyUrl('https://www.linkedin.com/in/john/') age=30 weight=70.5 married=True allergies=None contact_details={'phone': '123-456-7890'}
Data updated!


In [10]:
from pydantic import BaseModel, EmailStr, field_validator, model_validator
from typing import List, Dict


class Patient(BaseModel):
    name: str
    email: EmailStr
    age: int
    allergies: List[str]
    contact_details: Dict[str, str]

    @field_validator('name')
    @classmethod
    def transform_name(cls, value):
        return value.upper()

    @field_validator('age', mode='after')
    @classmethod
    def transform_age(cls, value):
        if 0 < value < 100:
            return value
        else:
            raise ValueError('Invalid age')

    @field_validator('email')
    @classmethod
    def email_validator(cls, value):
        valid_domains = ["hdfc.com", "icici.com", "sbi.com"]

        domain_name = value.split('@')[-1]

        if domain_name not in valid_domains:
            raise ValueError("Invalid domain name")

        return value

    @model_validator(mode="after")
    def validate_emergency_contact(cls, model):
        if model.age > 60 and "emergency" not in model.contact_details:
            raise ValueError("patient is older than 60 must have an emergency contact")

        return model


def insert_patient_data(patient: Patient):
    print(patient)
    print("Data inserted!")


def update_patient_data(patient: Patient):
    print(patient)
    print("Data updated!")


patient_info = {
    "name": "John",
    "email": "john319@icici.com",
    "age": "70",
    "weight": 70.5,
    "married": True,
    "allergies": ["peanut", "milk"],
    "contact_details": {"phone": "123-456-7890", "emergency": "987-654-3210"},
}

patient1 = Patient(**patient_info)

insert_patient_data(patient1)
update_patient_data(patient1)

name='JOHN' email='john319@icici.com' age=70 allergies=['peanut', 'milk'] contact_details={'phone': '123-456-7890', 'emergency': '987-654-3210'}
Data inserted!
name='JOHN' email='john319@icici.com' age=70 allergies=['peanut', 'milk'] contact_details={'phone': '123-456-7890', 'emergency': '987-654-3210'}
Data updated!
